$$
\huge \textsf{Project 2: interactive weather and global data visualization} \\
\LARGE \textsf{Cristina Aguilera, Jesús Antoñanzas} \\
\large \textsf{Information Visualization, UPC, GCED} \\
\large \textsf{December 22nd, 2019}
$$

# Part 1: Interactive Weather Visualization
## _pre-processing_

In [0]:
import pandas as pd
import altair as alt
import io
import datetime
from google.colab import files

In [0]:
# upload raw data 'weather.csv'
uploaded = files.upload()

Saving weather.csv to weather.csv


In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [0]:
weather = pd.read_csv(io.StringIO(uploaded['weather.csv'].decode('utf-8')))

The pre-processing in this part is quite simple. We just need to translate the date into months so filtering can be done and change some column names. There are no NA's in the variables we are going to use. 

In [0]:
# inspect data
weather.head()

,DATE,DailyAverageDryBulbTemperature,DailyAverageRelativeHumidity,DailyAverageSeaLevelPressure,DailyAverageStationPressure,DailyAverageWindSpeed,DailyCoolingDegreeDays,DailyHeatingDegreeDays,DailyDepartureFromNormalAverageTemperature,DailyMaximumDryBulbTemperature,DailySnowfall,DailyMinimumDryBulbTemperature,DailyPrecipitation,DailySustainedWindDirection,DailySustainedWindSpeed,MTD_PRCP,WindDirection
0,2018-01-01 23:59:00,-4,62.0,30.73,29.93,10.7,0,69,-28.4,1,0.0,-9,0.0,310,17,0.0,NW
1,2018-01-02 23:59:00,-1,62.0,30.48,29.62,11.1,0,66,-25.3,8,0.0,-9,0.0,220,22,0.0,SW
2,2018-01-03 23:59:00,12,65.0,30.15,29.37,12.3,0,53,-12.2,17,0.1,6,0.0,330,21,0.0,NW
3,2018-01-04 23:59:00,6,60.0,30.31,29.55,12.4,0,59,-18.1,12,0.0,0,0.0,330,20,0.0,NW
4,2018-01-05 23:59:00,5,60.0,30.45,29.68,10.8,0,60,-19.0,11,0.0,-1,0.0,320,20,0.0,NW


Create months variable so that they can be easily selected.





In [0]:
month = []
for i in range(len(weather)):
  month.append(datetime.datetime.strptime(weather["DATE"][i], '%Y-%m-%d %H:%M:%S').strftime('%B'))
weather["month"] = month

In [0]:
weather['month']

0       January
1       January
2       January
3       January
4       January
         ...   
360    December
361    December
362    December
363    December
364    December
Name: month, Length: 365, dtype: object

Change column names so code is shorter.

In [0]:
weather = weather.rename(columns = {'DATE': 'date', 'DailyAverageDryBulbTemperature': 'avg_temp', \
                                    'DailyMaximumDryBulbTemperature': 'max_temp', \
                                    'DailyMinimumDryBulbTemperature': 'min_temp', \
                                    'DailyPrecipitation': 'precipitation', 'DailySnowfall': 'snowfall', \
                                    'DailySustainedWindSpeed': 'Wind Speed (mph)', \
                                    'WindDirection': 'winddirection', \
                                    'DailySustainedWindDirection': 'winddirectiondegrees', \
                                    'DailyAverageRelativeHumidity': 'humidity'})
weather.columns

Index(['date', 'avg_temp', 'humidity', 'DailyAverageSeaLevelPressure',
       'DailyAverageStationPressure', 'DailyAverageWindSpeed',
       'DailyCoolingDegreeDays', 'DailyHeatingDegreeDays',
       'DailyDepartureFromNormalAverageTemperature', 'max_temp', 'snowfall',
       'min_temp', 'precipitation', 'winddirectiondegrees', 'Wind Speed (mph)',
       'MTD_PRCP', 'winddirection', 'month'],
      dtype='object')

And pre-processing is done. 

In [0]:
weather.to_csv('w_daily.csv')
files.download('w_daily.csv')

# Part 1: Interactive Weather Visualization
## _design_


In [3]:
# please upload clean data 'w_daily.csv'
uploaded = files.upload()
w_daily = pd.read_csv(io.StringIO(uploaded['w_daily.csv'].decode('utf-8')))

Saving w_daily.csv to w_daily.csv



We start on the most basic action: showing the average daily temperatures. 

In [0]:
# Average daily temperature
avg_t = alt.Chart(w_daily).mark_line(color = '#000000', strokeWidth = 1.7).encode(
    x = alt.X('date:T', title = 'Date'), 
    y = alt.Y('avg_temp:Q', title = 'Temperature (ºF)', scale = alt.Scale(domain = [-10, 100])),
)

In [5]:
avg_t

alt.Chart(...)

Now we add the option to select specific months.

In [0]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Dropdown box
month_dropdown = alt.binding_select(options = months)

# Selection tool
month_sel = alt.selection_single(fields = ['month'], bind = month_dropdown, name = '1.Select     ', clear = "dblclick")

In [7]:
avg_t.add_selection(month_sel).transform_filter(month_sel)

alt.Chart(...)

We can double click and it returns to an overview. Also notice how the 'Temperature' axis domain remains constant, which will help compare when we horizontally concatenate the plots. Let's now work on some details on demand.

In [0]:
# Hover selection
hover_sel = alt.selection(type="single", nearest=True, on="mouseover", empty = 'none', clear = "mouseout")

# Define what details to see
values = ['max_temp','avg_temp','min_temp'] 
ttip = [alt.Tooltip(value, type = "quantitative") for value in values]

# Selection grid
vlines = alt.Chart(w_daily).mark_rule().encode(
    x = "date:T",
    opacity = alt.condition(hover_sel, alt.value(0.4), alt.value(0)),
    tooltip = ttip
).add_selection(hover_sel)

# Points
points_avg = avg_t.mark_point().encode(
    opacity = alt.condition(hover_sel, alt.value(1), alt.value(0))
)

In [9]:
alt.layer(avg_t, vlines, points_avg).add_selection(month_sel).transform_filter(month_sel)

alt.LayerChart(...)

Let's now try to add some more variables: maximum and minimum daily temperatures, which can be toggles on and off with their respective radio buttons.

In [0]:
max_button = alt.binding_radio(options = [None, 'ON'])
min_button = alt.binding_radio(options = [None, 'ON'])

max_sel = alt.selection_single(fields = ['max_temp'], bind = max_button, name = "1.Show ", clear = "dblclick")
min_sel = alt.selection_single(fields = ['min_temp'], bind = min_button, name = "1.Show  ", clear = "dblclick")

# Maximum daily temperatures
max_t = alt.Chart(w_daily).mark_line(color = "#E3170A", strokeWidth = 1.3).encode(
    x = alt.X('date:T', title = "Date"), 
    y = alt.Y('max_temp:Q', title = "Temperature (ºF)"),
    opacity = alt.condition(max_sel, alt.value(0), alt.value(0.5))
).add_selection(max_sel)

# Minimum daily temperatures
min_t = alt.Chart(w_daily).mark_line(color = "#4472CA", strokeWidth = 1.3).encode(
    x = alt.X('date:T', title = "Date"), 
    y = alt.Y('min_temp:Q', title = "Temperature (ºF)"),
    opacity = alt.condition(min_sel, alt.value(0), alt.value(0.5))
).add_selection(min_sel)

In [11]:
alt.layer(avg_t, vlines, points_avg, max_t, min_t).add_selection(month_sel).transform_filter(month_sel)

alt.LayerChart(...)

Now we would like to also be able to see the maximum and minimum temperatures for each month, which we do with rules. We also add labels for better interpretability.

In [0]:
rule_button = alt.binding_radio(options = [None, 'ON'])
rule_sel = alt.selection_single(fields = ['rules'], bind = rule_button, name = "1.Max_min_avg", clear = "dblclick")

# Rule for the maximum temperature
month_max = alt.Chart(w_daily).mark_rule(color = 'firebrick').encode(
    y = 'max(avg_temp):Q',
    size = alt.SizeValue(1),
    opacity = alt.condition(rule_sel, alt.value(0), alt.value(1))
)

# Max text label
month_max_text = alt.Chart(w_daily).mark_text(dy = 7, dx = -135).encode(
    y = 'max(avg_temp):Q',
    text = 'max(avg_temp):Q',
    opacity = alt.condition(rule_sel, alt.value(0), alt.value(1))
)

# Rule for the minimum temperature
month_min = alt.Chart(w_daily).mark_rule(color = 'blue').encode(
    y='min(avg_temp):Q',
    size=alt.SizeValue(1),
    opacity = alt.condition(rule_sel, alt.value(0), alt.value(1))
)

# Min text label
month_min_text = alt.Chart(w_daily).mark_text(dy = 7, dx = -135).encode(
    y = 'min(avg_temp):Q',
    text = 'min(avg_temp):Q',
    opacity = alt.condition(rule_sel, alt.value(0), alt.value(1))
)

# Layer them
max_min_rules = alt.layer(month_max, month_max_text, month_min, month_min_text).add_selection(rule_sel)

In [13]:
alt.layer(avg_t, vlines, points_avg, max_t, min_t, max_min_rules).add_selection(month_sel).transform_filter(month_sel)

alt.LayerChart(...)

Notice how we have been using ```clear = 'dblclick'``` on all interactions. That is, if we want to clear all options added we can just double click on the plot and that will return us to the original plain overview. 

Now let's add some brush interacion on the 'x' axis. First we would like for the average temperature line to be highlighted when selected.

In [14]:
brush_sel = alt.selection(type = 'interval', encodings = ['x'], empty = 'none', clear = "dblclick")

# gray background with selection
background_avg = avg_t.encode(
    color = alt.value('#ddd')
)

# highlights on the transformed data
highlight_avg = avg_t.transform_filter(brush_sel)

avg_t = alt.layer(background_avg, highlight_avg)
alt.layer(avg_t, vlines, points_avg, max_t, min_t, max_min_rules).add_selection(brush_sel, month_sel).transform_filter(month_sel)

alt.LayerChart(...)

Notice that by default when nothing is selected an overview is still given. This aligns with the visualization mantra: "Overview first, zoom and filter, details on demand". We will accomplish all three while constructing the other interactions. 

Now, we would like to interactively see the average value in a certain time interval. 

In [0]:
# Interactive average line
avg_line = alt.Chart(w_daily).mark_rule(color = 'firebrick').encode(
    y = 'mean(avg_temp):Q',
    size = alt.SizeValue(2)
).transform_filter(brush_sel)

# Dynamic text label
avg_line_text = alt.Chart(w_daily).mark_text(dy = -7, dx = -135).encode(
    y = 'mean(avg_temp):Q',
    text = 'mean(avg_temp):Q',
).transform_filter(brush_sel)

avg_interact = alt.layer(avg_t, avg_line, avg_line_text)

In [16]:
alt.layer(avg_interact, vlines, points_avg, max_t, min_t, max_min_rules).add_selection(brush_sel).add_selection(month_sel).transform_filter(month_sel)

alt.LayerChart(...)

Let's add some more details on demand to this brush selector. We hihglight the area between the maximum and minimum temperatures on each day. 

In [0]:
# Area between avg_max's and avg_min's
temp_area = alt.Chart(w_daily).mark_area(opacity = 0).encode(
    x = 'date:T', 
    y = 'ci1(max_temp):Q',
    y2 = 'ci1(min_temp):Q'
)

# We have to construct a 'selected' area
selected = temp_area.mark_area(
    color = "#32A287",
    opacity = 0.5
).transform_filter(
    brush_sel
)

avg_area = alt.layer(temp_area, selected)

In [18]:
alt.layer(avg_interact, avg_area, vlines, points_avg, max_t, min_t, max_min_rules).transform_filter(month_sel).add_selection(month_sel, brush_sel)

alt.LayerChart(...)

We will now work on multiple charts. The biggest challenge here is cross-selection, which we'll adress later. We want to be able to select a time interval, which in turn would highlight some other variable in other plot/s.

We first add some replica plots to inspect different months at a time. Tooltips, hovers and radio buttons should not be shared in this case (because we want to independently adjust intervals and other selections), so we need to define all plots again with different interaction objects.

In [0]:
# Save the first one
temp_1 = alt.layer(avg_interact, avg_area, vlines, points_avg, max_t, min_t, max_min_rules).transform_filter(month_sel).add_selection(month_sel, brush_sel)

In [0]:
# Declare everything for the 2nd time

# Average daily temperature
avg_t_1 = alt.Chart(w_daily).mark_line(color = '#000000', strokeWidth = 1.7).encode(
    x = alt.X('date:T', title = "Date"), 
    y = alt.Y('avg_temp:Q', title = "Temperature (ºF)", scale = alt.Scale(domain = [-10, 100]))
)

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Dropdown box
month_dropdown_1 = alt.binding_select(options = months)

# Selection tool
month_sel_1 = alt.selection_single(fields = ['month'], bind = month_dropdown_1, name = '2.Select     ', clear = "dblclick")

hover_sel_1 = alt.selection(type="single", nearest=True, on="mouseover", empty = 'none', clear = "mouseout")

# Define what details to see
values = ['max_temp','avg_temp','min_temp'] 
ttip = [alt.Tooltip(value, type = "quantitative") for value in values]

# Selection grid
vlines_1 = alt.Chart(w_daily).mark_rule().encode(
    x = "date:T",
    opacity = alt.condition(hover_sel_1, alt.value(0.4), alt.value(0)),
    tooltip = ttip
).add_selection(hover_sel_1)

# Points
points_avg_1 = avg_t_1.mark_point().encode(
    opacity = alt.condition(hover_sel_1, alt.value(1), alt.value(0))
)

max_button_1 = alt.binding_radio(options = [None, 'ON'])
min_button_1 = alt.binding_radio(options = [None, 'ON'])

max_sel_1 = alt.selection_single(fields = ['max_temp'], bind = max_button_1, name = "2.Show ", clear = "dblclick")
min_sel_1 = alt.selection_single(fields = ['min_temp'], bind = min_button_1, name = "2.Show  ", clear = "dblclick")

# Maximum daily temperatures
max_t_1 = alt.Chart(w_daily).mark_line(color = "#E3170A", strokeWidth = 1.3).encode(
    x = alt.X('date:T', title = "Date"), 
    y = alt.Y('max_temp:Q', title = "Temperature (ºF)"),
    opacity = alt.condition(max_sel_1, alt.value(0), alt.value(0.5))
).add_selection(max_sel_1)

# Minimum daily temperatures
min_t_1 = alt.Chart(w_daily).mark_line(color = "#4472CA", strokeWidth = 1.3).encode(
    x = alt.X('date:T', title = "Date"), 
    y = alt.Y('min_temp:Q', title = "Temperature (ºF)"),
    opacity = alt.condition(min_sel_1, alt.value(0), alt.value(0.5))
).add_selection(min_sel_1)

rule_button_1 = alt.binding_radio(options = [None, 'ON'])
rule_sel_1 = alt.selection_single(fields = ['rules'], bind = rule_button_1, name = "2.Max_min_avg", clear = "dblclick")

# Rule for the maximum temperature
month_max_1 = alt.Chart(w_daily).mark_rule(color = 'firebrick').encode(
    y = 'max(avg_temp):Q',
    size = alt.SizeValue(1),
    opacity = alt.condition(rule_sel_1, alt.value(0), alt.value(1))
)

# Max text label
month_max_text_1 = alt.Chart(w_daily).mark_text(dy = 7, dx = -135).encode(
    y = 'max(avg_temp):Q',
    text = 'max(avg_temp):Q',
    opacity = alt.condition(rule_sel_1, alt.value(0), alt.value(1))
)

# Rule for the minimum temperature
month_min_1 = alt.Chart(w_daily).mark_rule(color = 'blue').encode(
    y='min(avg_temp):Q',
    size=alt.SizeValue(1),
    opacity = alt.condition(rule_sel_1, alt.value(0), alt.value(1))
)

# Min text label
month_min_text_1 = alt.Chart(w_daily).mark_text(dy = 7, dx = -135).encode(
    y = 'min(avg_temp):Q',
    text = 'min(avg_temp):Q',
    opacity = alt.condition(rule_sel_1, alt.value(0), alt.value(1))
)

# Layer them
max_min_rules_1 = alt.layer(month_max_1, month_max_text_1, month_min_1, month_min_text_1).add_selection(rule_sel_1)

brush_sel_1 = alt.selection(type = 'interval', encodings = ['x'], empty = 'none', clear = "dblclick")

# gray background with selection
background_avg_1 = avg_t_1.encode(
    color = alt.value('#ddd')
)

# highlights on the transformed data
highlight_avg_1 = avg_t_1.transform_filter(brush_sel_1)

# overwrite average line plot
avg_t_1 = alt.layer(background_avg_1, highlight_avg_1)

# Interactive average line
avg_line_1 = alt.Chart(w_daily).mark_rule(color = 'firebrick').encode(
    y = 'mean(avg_temp):Q',
    size = alt.SizeValue(2)
).transform_filter(brush_sel_1)

# Dynamic text label
avg_line_text_1 = alt.Chart(w_daily).mark_text(dy = -7, dx = -135).encode(
    y = 'mean(avg_temp):Q',
    text = 'mean(avg_temp):Q',
).transform_filter(brush_sel_1)

avg_interact_1 = alt.layer(avg_t_1, avg_line_1, avg_line_text_1)

# Area between avg_max's and avg_min's
temp_area_1 = alt.Chart(w_daily).mark_area(opacity = 0).encode(
    x = 'date:T', 
    y = 'ci1(max_temp):Q',
    y2 = 'ci1(min_temp):Q'
)

# We have to construct a 'selected' area
selected_1 = temp_area.mark_area(
    color = "#32A287",
    opacity = 0.5
).transform_filter(
    brush_sel_1
)

avg_area_1 = alt.layer(temp_area_1, selected_1)

In [0]:
# And save it
temp_2 = alt.layer(avg_interact_1, avg_area_1, vlines_1, points_avg_1, max_t_1, min_t_1, max_min_rules_1).transform_filter(month_sel_1).add_selection(month_sel_1, brush_sel_1)

In [22]:
temp_1 | temp_2

alt.HConcatChart(...)

And a third time

In [0]:
# Declare everything for the 2nd time

# Average daily temperature
avg_t_2 = alt.Chart(w_daily).mark_line(color = '#000000', strokeWidth = 1.7).encode(
    x = alt.X('date:T', title = "Date"), 
    y = alt.Y('avg_temp:Q', title = "Temperature (ºF)", scale = alt.Scale(domain = [-10, 100]))
)

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Dropdown box
month_dropdown_2 = alt.binding_select(options = months)

# Selection tool
month_sel_2 = alt.selection_single(fields = ['month'], bind = month_dropdown_2, name = '3.Select     ', clear = "dblclick")

hover_sel_2 = alt.selection(type="single", nearest=True, on="mouseover", empty = 'none', clear = "mouseout")

# Define what details to see
values = ['max_temp','avg_temp','min_temp'] 
ttip = [alt.Tooltip(value, type = "quantitative") for value in values]

# Selection grid
vlines_2 = alt.Chart(w_daily).mark_rule().encode(
    x = "date:T",
    opacity = alt.condition(hover_sel_2, alt.value(0.4), alt.value(0)),
    tooltip = ttip
).add_selection(hover_sel_2)

# Points
points_avg_2 = avg_t_2.mark_point().encode(
    opacity = alt.condition(hover_sel_2, alt.value(1), alt.value(0))
)

max_button_2 = alt.binding_radio(options = [None, 'ON'])
min_button_2 = alt.binding_radio(options = [None, 'ON'])

max_sel_2 = alt.selection_single(fields = ['max_temp'], bind = max_button_2, name = "3.Show ", clear = "dblclick")
min_sel_2 = alt.selection_single(fields = ['min_temp'], bind = min_button_2, name = "3.Show  ", clear = "dblclick")

# Maximum daily temperatures
max_t_2 = alt.Chart(w_daily).mark_line(color = "#E3170A", strokeWidth = 1.3).encode(
    x = alt.X('date:T', title = "Date"), 
    y = alt.Y('max_temp:Q', title = "Temperature (ºF)"),
    opacity = alt.condition(max_sel_2, alt.value(0), alt.value(0.5))
).add_selection(max_sel_2)

# Minimum daily temperatures
min_t_2 = alt.Chart(w_daily).mark_line(color = "#4472CA", strokeWidth = 1.3).encode(
    x = alt.X('date:T', title = "Date"), 
    y = alt.Y('min_temp:Q', title = "Temperature (ºF)"),
    opacity = alt.condition(min_sel_2, alt.value(0), alt.value(0.5))
).add_selection(min_sel_2)

rule_button_2 = alt.binding_radio(options = [None, 'ON'])
rule_sel_2 = alt.selection_single(fields = ['rules'], bind = rule_button_2, name = "3.Max_min_avg", clear = "dblclick")

# Rule for the maximum temperature
month_max_2 = alt.Chart(w_daily).mark_rule(color = 'firebrick').encode(
    y = 'max(avg_temp):Q',
    size = alt.SizeValue(1),
    opacity = alt.condition(rule_sel_2, alt.value(0), alt.value(1))
)

# Max text label
month_max_text_2 = alt.Chart(w_daily).mark_text(dy = 7, dx = -135).encode(
    y = 'max(avg_temp):Q',
    text = 'max(avg_temp):Q',
    opacity = alt.condition(rule_sel_2, alt.value(0), alt.value(1))
)

# Rule for the minimum temperature
month_min_2 = alt.Chart(w_daily).mark_rule(color = 'blue').encode(
    y='min(avg_temp):Q',
    size=alt.SizeValue(1),
    opacity = alt.condition(rule_sel_2, alt.value(0), alt.value(1))
)

# Min text label
month_min_text_2 = alt.Chart(w_daily).mark_text(dy = 7, dx = -135).encode(
    y = 'min(avg_temp):Q',
    text = 'min(avg_temp):Q',
    opacity = alt.condition(rule_sel_2, alt.value(0), alt.value(1))
)

# Layer them
max_min_rules_2 = alt.layer(month_max_2, month_max_text_2, month_min_2, month_min_text_2).add_selection(rule_sel_2)

brush_sel_2 = alt.selection(type = 'interval', encodings = ['x'], empty = 'none', clear = "dblclick")

# gray background with selection
background_avg_2 = avg_t_2.encode(
    color = alt.value('#ddd')
)

# highlights on the transformed data
highlight_avg_2 = avg_t_2.transform_filter(brush_sel_2)

# overwrite average line plot
avg_t_2 = alt.layer(background_avg_2, highlight_avg_2)

# Interactive average line
avg_line_2 = alt.Chart(w_daily).mark_rule(color = 'firebrick').encode(
    y = 'mean(avg_temp):Q',
    size = alt.SizeValue(2)
).transform_filter(brush_sel_2)

# Dynamic text label
avg_line_text_2 = alt.Chart(w_daily).mark_text(dy = -7, dx = -135).encode(
    y = 'mean(avg_temp):Q',
    text = 'mean(avg_temp):Q',
).transform_filter(brush_sel_2)

avg_interact_2 = alt.layer(avg_t_2, avg_line_2, avg_line_text_2)

# Area between avg_max's and avg_min's
temp_area_2 = alt.Chart(w_daily).mark_area(opacity = 0).encode(
    x = 'date:T', 
    y = 'ci1(max_temp):Q',
    y2 = 'ci1(min_temp):Q'
)

# We have to construct a 'selected' area
selected_2 = temp_area.mark_area(
    color = "#32A287",
    opacity = 0.5
).transform_filter(
    brush_sel_2
)

avg_area_2 = alt.layer(temp_area_2, selected_2)

In [0]:
# And save it
temp_3 = alt.layer(avg_interact_2, avg_area_2, vlines_2, points_avg_2, max_t_2, min_t_2, max_min_rules_2).transform_filter(month_sel_2).add_selection(month_sel_2, brush_sel_2)

In [25]:
temp_1 | temp_2 | temp_3

alt.HConcatChart(...)

Notice that double click restores all plots. Now let's try adding plots with which to cross-interact. First, we would like to show wind directions and speeds, with the ability to select different wind directions. 

In [0]:
# Interval selector on the 'x' axis
brush_selector = alt.selection(type = 'interval', encodings = ['x'], empty = 'none', clear = "dblclick")

# Define what details the tooltpi will show
ttip_wind = alt.Tooltip('count(winddirectiondegrees)', type = "quantitative")

# Base chart 
wind = alt.Chart(w_daily).mark_bar().encode(
    x = alt.X('winddirectiondegrees:Q', bin=alt.Bin(step=30), title = 'Wind Direction (degrees)'),
    y = 'count(winddirectiondegrees):Q',
    color = alt.Color("Wind Speed (mph):Q", bin=alt.Bin(step=5), \
                      scale=alt.Scale(scheme='yellowgreenblue'), \
                      legend = alt.Legend(orient = 'left', titleAlign = 'center')),
    tooltip = ttip_wind
)

# gray background with selection
background_wind = wind.encode(
    color=alt.value('#ddd')
)

# highlights on the transformed data
highlight_wind = wind.transform_filter(brush_selector)

wind_layer = alt.layer(background_wind, highlight_wind)

We would also like to see the value of the proportions of wind speeds for each 'wind direction' bin.  

In [27]:
# base text
windspeed_text = alt.Chart(w_daily).mark_text(dx = 0, dy = 6, color = 'white').encode(
    x = alt.X('winddirectiondegrees:Q', bin = alt.Bin(step = 30)),
    y = alt.Y('count(winddirectiondegrees):Q', stack = 'zero'),
    detail = alt.Detail('Wind Speed (mph):N', bin=alt.Bin(step = 5)),
    text = alt.Text('count(winddirectiondegrees)', format = '.1f')
)

# '00' at the end of the hex code means full transparency (in browsers),
# as hex code is #RRGGBBAA, where AA is the alpha value transparency
windspeed_bg = windspeed_text.encode(
    color = alt.value('#aaaaaa00')
)

# highlight
windspeed_hl = windspeed_text.transform_filter(brush_selector)

w_text = alt.layer(windspeed_bg, windspeed_hl)

# final wind plot
wind_int = alt.layer(wind_layer, w_text)
wind_int.add_selection(month_sel, brush_selector).transform_filter(month_sel)

alt.LayerChart(...)

The decission to put the values in degrees and not Cardinal Directions (North, East, West, South) is justified: The latter are nominal values, so when the brush touches a bin it will all be selected. On the other hand when using degrees interval selection is more accurate. In order to help imagine, one can think of a circle with the Cardinal Directions labeled and unroll it to the right. The equivalent would be our 'degree' axis.

Now let's add an interactive precipitation plot, which will also allow to see snowfall quantities when hovering. 

In [0]:
# Precipitation buttons, precipitation is 'ON' by default
prec_button = alt.binding_radio(options = [None, 'ON'])
prec_sel = alt.selection(fields = ['precipitation'], type = 'single', \
                         name = "Show precipitation vals", bind = prec_button, \
                         init = {'precipitation': 'ON'})

# Interactive precipitation plot
precip_plot = alt.Chart(w_daily).mark_bar(color = 'firebrick').encode(
    x = alt.X('date:T', title = 'Date'),
    y = alt.Y('precipitation:Q', title = 'Precipitation (mm)'),
    opacity = alt.condition(prec_sel, alt.value(0), alt.value(1))
)

# Add background
background_prec = precip_plot.encode(
    color=alt.value('#ddd')
)

# And highlights
highlight_prec = precip_plot.transform_filter(brush_selector)

precipitation = alt.layer(background_prec, highlight_prec).add_selection(prec_sel)

# With hovering interaction
hover_prec = alt.selection(type="single", nearest=True, on="mouseover", empty = 'none', clear = "mouseout")

# Define what details to see
values = ['precipitation', 'snowfall'] 
ttip_prec = [alt.Tooltip(value, type = "quantitative") for value in values]

# Selection grid
vlines_prec = alt.Chart(w_daily).mark_rule().encode(
    x = "date:T",
    opacity = alt.condition(hover_prec, alt.value(0.4), alt.value(0)),
    tooltip = ttip_prec
).add_selection(hover_prec)

precip = alt.layer(precipitation, vlines_prec)

To which we add the option to display snowfall values

In [29]:
snowfall_button = alt.binding_radio(options = [None, 'ON'])
snowfall_sel = alt.selection(type = 'single', name = "Show snow", bind = snowfall_button)

# Base plot
snow_int = alt.Chart(w_daily).mark_bar().encode(
    x = 'date:T',
    y = alt.Y('snowfall:Q', title = 'Snowfall (in)'),
    opacity = alt.condition(snowfall_sel, alt.value(0), alt.value(1))
)

# Background
bg_snow = snow_int.encode(
    color = alt.value('#ddd')
)

# highlights
hl_snow = snow_int.transform_filter(brush_selector)

snow = alt.layer(bg_snow, hl_snow).add_selection(snowfall_sel)

precip_int = alt.layer(precip, snow).resolve_scale(y = 'independent')
precip_int.add_selection(brush_selector)

alt.LayerChart(...)

We add the month selection and we share the brush selectors, the result being a cross-interactive plot. 

In [0]:
month_sel_3 = alt.selection_single(fields = ['month'], bind = month_dropdown_2, name = '4.Select', clear = "dblclick")

In [31]:
# adding selections and transform filters as a common factor will create bugs when concatenating temperature plots
(wind_int.add_selection(month_sel_3, brush_selector).transform_filter(month_sel_3) | precip_int.add_selection(month_sel_3, brush_selector).transform_filter(month_sel_3))

alt.HConcatChart(...)

We can then put all plots together.

In [32]:
(wind_int.add_selection(month_sel_3, brush_selector).transform_filter(month_sel_3) | \
 precip_int.add_selection(month_sel_3, brush_selector).transform_filter(month_sel_3)) & \
 (temp_1 | temp_2 | temp_3)

alt.VConcatChart(...)

Now, we would like the first temperature plot to be linked with wind and precipitation. This can be achieved if they use the same brush and month selectors. First, though, we know that when wind will be selected, the temperature plot (temporal values) will behave wierdly as it has 'line' marks. So, we will create another, more simple, temperature plot. It will just contain daily average temperatures, dynamic mean temperatures and a tooltip, and it will be made up of points, not lines. Also, we can add humidity measures. 

In [33]:
# Declare temperature, but removing some things

avg_button = alt.binding_radio(options = [None, 'ON'])
avg_sel = alt.selection(fields = ['avg_temp'], type = 'single', \
                        name = "Cross show   ", bind = avg_button, \
                        init = {'avg_temp': 'ON'})

# Average daily temperature
avg_t_3 = alt.Chart(w_daily).mark_point(color = '#000000', strokeWidth = 1.7).encode(
    x = alt.X('date:T', title = "Date"), 
    y = alt.Y('avg_temp:Q', title = "Temperature (ºF)", scale = alt.Scale(domain = [-10, 100])),
    opacity = alt.condition(avg_sel, alt.value(0), alt.value(1))
)

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Dropdown box
month_dropdown_3 = alt.binding_select(options = months)

# Selection tool
month_sel_3 = alt.selection_single(fields = ['month'], bind = month_dropdown_3, name = 'Cross select    ', clear = "dblclick")

hover_sel_3 = alt.selection(type="single", nearest=True, on="mouseover", empty = 'none', clear = "mouseout")

# Define what details to see
values = ['max_temp','avg_temp','min_temp', 'humidity'] 
ttip = [alt.Tooltip(value, type = "quantitative") for value in values]

# Selection grid
vlines_3 = alt.Chart(w_daily).mark_rule().encode(
    x = "date:T",
    opacity = alt.condition(hover_sel_3, alt.value(0.4), alt.value(0)),
    tooltip = ttip
).add_selection(hover_sel_3)

rule_button_3 = alt.binding_radio(options = [None, 'ON'])
rule_sel_3 = alt.selection_single(fields = ['rules'], bind = rule_button_3, name = "Cross max_min_avg", clear = "dblclick")

# Rule for the maximum temperature
month_max_3 = alt.Chart(w_daily).mark_rule(color = 'firebrick').encode(
    y = 'max(avg_temp):Q',
    size = alt.SizeValue(1),
    opacity = alt.condition(rule_sel_3, alt.value(0), alt.value(1))
)

# Max text label
month_max_text_3 = alt.Chart(w_daily).mark_text(dy = 7, dx = -135).encode(
    y = 'max(avg_temp):Q',
    text = 'max(avg_temp):Q',
    opacity = alt.condition(rule_sel_3, alt.value(0), alt.value(1))
)

# Rule for the minimum temperature
month_min_3 = alt.Chart(w_daily).mark_rule(color = 'blue').encode(
    y='min(avg_temp):Q',
    size=alt.SizeValue(1),
    opacity = alt.condition(rule_sel_3, alt.value(0), alt.value(1))
)

# Min text label
month_min_text_3 = alt.Chart(w_daily).mark_text(dy = 7, dx = -135).encode(
    y = 'min(avg_temp):Q',
    text = 'min(avg_temp):Q',
    opacity = alt.condition(rule_sel_3, alt.value(0), alt.value(1))
)

# Layer them
max_min_rules_3 = alt.layer(month_max_3, month_max_text_3, month_min_3, month_min_text_3).add_selection(rule_sel_3)

brush_sel_3 = alt.selection(type = 'interval', encodings = ['x'], empty = 'none', clear = "dblclick")

# gray background with selection
background_avg_3 = avg_t_3.encode(
    color = alt.value('#ddd')
)

# highlights on the transformed data
highlight_avg_3 = avg_t_3.transform_filter(brush_sel_3)

# overwrite average plot
avg_t_3 = alt.layer(background_avg_3, highlight_avg_3)

# Interactive average line
avg_line_3 = alt.Chart(w_daily).mark_rule(color = 'firebrick').encode(
    y = 'mean(avg_temp):Q',
    size = alt.SizeValue(2),
    opacity = alt.condition(avg_sel, alt.value(0), alt.value(1))
).transform_filter(brush_sel_3)

# Dynamic text label
avg_line_text_3 = alt.Chart(w_daily).mark_text(dy = -7, dx = -135).encode(
    y = 'mean(avg_temp):Q',
    text = 'mean(avg_temp):Q',
    opacity = alt.condition(avg_sel, alt.value(0), alt.value(1))
).transform_filter(brush_sel_3)

avg_interact_3 = alt.layer(avg_t_3, avg_line_3, avg_line_text_3)

temp_cross = alt.layer(avg_interact_3, vlines_3, max_min_rules_3)
temp_cross.transform_filter(month_sel_3).add_selection(month_sel_3, brush_sel_3, avg_sel)

alt.LayerChart(...)

To which we add the option to see humidity points, with also dynamic average.

In [34]:
humidity_button = alt.binding_radio(options = [None, 'ON'])
humidity_sel = alt.selection(fields = ['humidity'], type = 'single', name = "Cross show    ", bind = humidity_button)

# Base plot
hum_int = alt.Chart(w_daily).mark_point().encode(
    x = 'date:T',
    y = alt.Y('humidity:Q', title = 'Relative humidity (%)'),
    opacity = alt.condition(humidity_sel, alt.value(0), alt.value(1))
)

# Background
bg_hum = hum_int.encode(
    color = alt.value('#ddd')
)

# highlights
hl_hum = hum_int.transform_filter(brush_sel_3)

# Interactive average humidity line
hum_line = alt.Chart(w_daily).mark_rule(color = 'lightblue').encode(
    y = 'mean(humidity):Q',
    size = alt.SizeValue(2),
    opacity = alt.condition(avg_sel, alt.value(0), alt.value(1))
).transform_filter(brush_sel_3)

# Dynamic text label
hum_line_text = alt.Chart(w_daily).mark_text(dy = -7, dx = 135).encode(
    y = 'mean(humidity):Q',
    text = 'mean(humidity):Q',
    opacity = alt.condition(avg_sel, alt.value(0), alt.value(1))
).transform_filter(brush_sel_3)

hum = alt.layer(bg_hum, hl_hum, hum_line, hum_line_text)

temp_hum_cross = (alt.layer(temp_cross, hum).transform_filter(month_sel_3)
                     .add_selection(month_sel_3, brush_sel_3, avg_sel, humidity_sel)
                     .resolve_scale(y = 'independent'))
temp_hum_cross

alt.LayerChart(...)

We don't add points showing where the hovering line is because when we deactivate humidity or temperature they will appear floating on top of nothing, depending on which variable we are emphasizing.

Now, we re-declare wind and precipitation plots so that they share month and brush selectors. We also add dynamic average rules for precipitation and snowfall.

In [0]:
# Reminder: the brush used in 'temp_simple' was 'brush_sel_3', and the month dropdown was 'month_sel_3'.

# WIND

# Define what details the tooltip will show
ttip_wind = alt.Tooltip('count(winddirectiondegrees)', type = "quantitative")

wind = alt.Chart(w_daily).mark_bar().encode(
    x = alt.X("winddirectiondegrees:Q", bin=alt.Bin(step=30), title = 'Wind Direction (degrees)'),
    y = "count(winddirectiondegrees):Q",
    color = alt.Color("Wind Speed (mph):Q", bin=alt.Bin(step=5), \
                      scale=alt.Scale(scheme='yellowgreenblue'), \
                      legend = alt.Legend(orient = 'left', titleAlign = 'center')),
    tooltip = ttip_wind
)

# gray background with selection
background_wind = wind.encode(
    color=alt.value('#ddd')
)

# highlights on the transformed data
highlight_wind = wind.transform_filter(brush_sel_3)

wind_layer = alt.layer(background_wind, highlight_wind)

# base text
windspeed_text = alt.Chart(w_daily).mark_text(dx = 0, dy = 6, color = 'white').encode(
    x = alt.X('winddirectiondegrees:Q', bin = alt.Bin(step = 30)),
    y = alt.Y('count(winddirectiondegrees):Q', stack = 'zero'),
    detail = alt.Detail('Wind Speed (mph):N', bin=alt.Bin(step = 5)),
    text = alt.Text('count(winddirectiondegrees)', format = '.1f')
)

# '00' at the end of the hex code means full transparency (in browsers),
# as hex code is #RRGGBBAA, where AA is the alpha value transparency
windspeed_bg = windspeed_text.encode(
    color = alt.value('#dddddd00')
)

# highlight
windspeed_hl = windspeed_text.transform_filter(brush_sel_3)

w_text = alt.layer(windspeed_bg, windspeed_hl)

# final wind plot
wind_cross = alt.layer(wind_layer, w_text).add_selection(brush_sel_3)

# PRECIPITATION

# Precipitation buttons, precipitation is 'ON' by default
prec_button_1 = alt.binding_radio(options = [None, 'ON'])
prec_sel_1 = alt.selection(fields = ['precipitation'], type = 'single', \
                           name = "Cross show ", bind = prec_button_1, \
                           init = {'precipitation': 'ON'})

# Interactive precipitation plot
precip_plot = alt.Chart(w_daily).mark_bar(color = '#4A6583').encode(
    x = alt.X('date:T', title = 'Date'),
    y = alt.Y('precipitation:Q', title = 'Precipitation (mm)'),
    opacity = alt.condition(prec_sel_1, alt.value(0), alt.value(1))
)

# Add background
background_prec = precip_plot.encode(
    color=alt.value('#ddd')
)

# And highlights
highlight_prec = precip_plot.transform_filter(brush_sel_3)

precipitation = alt.layer(background_prec, highlight_prec)

# With hovering interaction
hover_prec = alt.selection(type="single", nearest=True, on = "mouseover", \
                           empty = 'none', clear = "mouseout")

# Define what details to see
values = ['precipitation', 'snowfall']
ttip_prec = [alt.Tooltip(value, type = "quantitative") for value in values]

# Selection grid
vlines_prec = alt.Chart(w_daily).mark_rule().encode(
    x = "date:T",
    opacity = alt.condition(hover_prec, alt.value(0.4), alt.value(0)),
    tooltip = ttip_prec
).add_selection(hover_prec)

# Interactive average precipitation line
prec_line = alt.Chart(w_daily).mark_rule(color = '#4A6583').encode(
    y = 'mean(precipitation):Q',
    size = alt.SizeValue(2),
    opacity = alt.condition(avg_sel, alt.value(0), alt.value(1))
).transform_filter(brush_sel_3)

# Dynamic text label
prec_line_text = alt.Chart(w_daily).mark_text(dy = -7, dx = -135).encode(
    y = 'mean(precipitation):Q',
    text = 'mean(precipitation):Q',
    opacity = alt.condition(avg_sel, alt.value(0), alt.value(1))
).transform_filter(brush_sel_3)

precip = alt.layer(precipitation, vlines_prec, prec_line, prec_line_text).add_selection(brush_sel_3, prec_sel_1)

# SNOWFALL

snowfall_button_1 = alt.binding_radio(options = [None, 'ON'])
snowfall_sel_1 = alt.selection(fields = ['snow'], type = 'single', \
                               name = "Cross show       ", bind = snowfall_button)

# Base plot
snow_int = alt.Chart(w_daily).mark_bar(color = '#ABABAB').encode(
    x = 'date:T',
    y = alt.Y('snowfall:Q', title = 'Snowfall (in)'),
    opacity = alt.condition(snowfall_sel_1, alt.value(0), alt.value(1))
)

# Background
bg_snow = snow_int.encode(
    color = alt.value('#ddd')
)

# highlights
hl_snow = snow_int.transform_filter(brush_sel_3)

# Interactive average snowfall line
snow_line = alt.Chart(w_daily).mark_rule(color = '#ABABAB').encode(
    y = 'mean(snowfall):Q',
    size = alt.SizeValue(2),
    opacity = alt.condition(avg_sel, alt.value(0), alt.value(1))
).transform_filter(brush_sel_3)

# Dynamic text label
snow_line_text = alt.Chart(w_daily).mark_text(dy = -7, dx = 135).encode(
    y = 'mean(snowfall):Q',
    text = 'mean(snowfall):Q',
    opacity = alt.condition(avg_sel, alt.value(0), alt.value(1))
).transform_filter(brush_sel_3)

snow = alt.layer(snow_line, snow_line_text, bg_snow, hl_snow)

precip_snow_cross = (alt.layer(precip, snow)
                        .add_selection(brush_sel_3, snowfall_sel_1)
                        .resolve_scale(y = 'independent'))

And we are done. These are our definitive plots. We would have liked to be able to configure the dual 'y' axis colors, but it cannot be done for each individual plot if later it is going to be used in 'hconcat' or 'vconcat', which is our case. One thing to have in mind: the 'wind speed' legend binning adjusts automatically depending on the month selected (increases binning step), but does not go back to its original configuration, so one might need to restart the plot.

Of course, they can be shown together, but interaction with buttons is cumbersome because of their placement and some bug may appear, so it is more recommended to use them separately (as shown below).

In [41]:
alt.vconcat(alt.hconcat(wind_cross.add_selection(month_sel_3).transform_filter(month_sel_3), \
                        precip_snow_cross.add_selection(month_sel_3).transform_filter(month_sel_3), \
                        temp_hum_cross), \
            alt.hconcat(temp_1, temp_2, temp_3), \
            title = alt.TitleParams('2018 Interactive Chicago Weather visualization', \
                                    fontSize = 20, anchor = 'middle'))

alt.VConcatChart(...)

Here are the independent dynamic temperature plots.

In [37]:
alt.hconcat(temp_1, temp_2, temp_3, \
            title = alt.TitleParams('2018 Interactive Chicago Weather visualization: dynamic comparison', \
                                    fontSize = 20, anchor = 'middle'))

alt.HConcatChart(...)

And the cross interactive plots, which can be shown separate to the other three temperature plots so the button section is less crowded and because they do not depend on them.

In [42]:
(alt.hconcat(wind_cross.add_selection(month_sel_3).transform_filter(month_sel_3), \
            precip_snow_cross.add_selection(month_sel_3).transform_filter(month_sel_3), \
            temp_hum_cross, \
            title = alt.TitleParams('2018 Interactive Chicago Weather visualization: cross interaction', \
                                    fontSize = 20, anchor = 'middle')))

alt.HConcatChart(...)